# Initial Test of Facial Keypoint Detector
## Without PCA and an SVM for each class

First we need to identify the keypoint detector model that we will be using. 

MTCNN - Tensorflow
https://github.com/ipazc/mtcnn
(Appears to be pretrained)
It appears to provide a few facial keypoints. May be useful for patch based approach probably not for direct-point position approach.

Pytorch-Facial-Keypoints - Pytorch
https://github.com/fiyero/pytorch_facial_keypoints
(Not pretrained)
Generates keypoints around face but excludes ears. Can be used for direct-point position approach.

Openface - in Lua?
https://github.com/cmusatyalab/openface
(Might be pretrained)
Not sure if it generates keypoints

Openface - 
https://github.com/TadasBaltrusaitis/OpenFace
(Pretrained models)
High number of facial points. Can be used for direct-point position approach.

Facial-Landmark
https://github.com/huohuotm/Facial-Landmark
(pretrained models)
High number of facial points. Can be used for direct-point position approach.

Face-Recognition
https://github.com/ageitgey/face_recognition
(Is pretrained and comes as a library)
Detects facial points.

## Preprocessing images
TODO ?

## Processing on keypoints
TODO ?

## For the classifier
https://scikit-learn.org/stable/modules/svm.html
Recommend Linear SVC first

# Second approach is an end-to-end solution

Something akin to https://github.com/ZER-0-NE/ML_problems